In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2024-04-06 15:33:32.300283: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 15:33:32.305130: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 15:33:32.402852: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 15:33:32.404443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-04-06 15:33:33.740834: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [14]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [15]:
index_to_save = 4

<IPython.core.display.Javascript object>

In [16]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = [
    "Cement_Type_CP III32",
    "Cement_Type_CP VARI",
    "Cement_Type_Fibrocimento",
]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [17]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "203",
    "Plant": "AV",
    "Features": "Chemical + CS3",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [18]:
df = pd.read_csv("../../../../../../data/processed/203/av.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [19]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",
        "#200",
        "#325",
        "Final setting time",
        "Initial setting time",
        "CS1",
        "CS7",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [20]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [21]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [22]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2024-04-06 15:33:39.161117: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-06 15:33:39.161201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-04-06 15:33:39.161219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-04-06 15:33:39.161539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.147.5
2024-04-06 15:33:39.161594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.147.5
2024-04-06 15:33:39.161602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.147.5


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.047 (0.040)
MAE: 1.623 (0.034)
MAPE: 0.033 (0.001)
R2: 0.745 (0.013)


******
[TEST]
******
RMSE: 2.156 (0.115)
MAE: 1.695 (0.080)
MAPE: 0.034 (0.002)
R2: 0.714 (0.038)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.666 (0.122)
MAE: 1.314 (0.097)
MAPE: 0.026 (0.002)
R2: 0.833 (0.028)


******
[TEST]
******
RMSE: 2.122 (0.193)
MAE: 1.631 (0.155)
MAPE: 0.033 (0.003)
R2: 0.727 (0.045)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.867 (0.180)
MAE: 1.512 (0.163)
MAPE: 0.030 (0.003)
R2: 0.790 (0.039)


******
[TEST]
******
RMSE: 2.544 (0.123)
MAE: 1.995 (0.099)
MAPE: 0.040 (0.002)
R2: 0.577 (0.069)


Minutes Elapsed:  81.38248473008474


<IPython.core.display.Javascript object>

In [23]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model Cross Validation Timesteps RMSE Test            \
                                                        mean       std   
0  Chemical + CS3  LSTM   Repeated KFold         1  2.156191  0.115033   
1  Chemical + CS3  LSTM   Repeated KFold         7  2.121615  0.193009   
2  Chemical + CS3  LSTM   Repeated KFold        14  2.544401  0.123213   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  1.694980  0.079981  0.034336  0.001540  0.714076  0.038272  
1  1.631287  0.154577  0.033145  0.002986  0.726855  0.044942  
2  1.995277  0.099470  0.039514  0.001851  0.577271  0.068931

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [24]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 5.787 (1.170)
MAE: 4.324 (0.850)
MAPE: 0.086 (0.015)
R2: -11.273 (7.641)


******
[TEST]
******
RMSE: 15.331 (13.806)
MAE: 11.593 (10.342)
MAPE: 0.244 (0.230)
R2: -65.582 (116.275)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.940 (0.548)
MAE: 1.591 (0.512)
MAPE: 0.032 (0.010)
R2: -0.256 (0.847)


******
[TEST]
******
RMSE: 10.367 (12.788)
MAE: 7.972 (9.574)
MAPE: 0.173 (0.213)
R2: -37.863 (79.632)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 2.043 (0.497)
MAE: 1.705 (0.475)
MAPE: 0.034 (0.008)
R2: -0.717 (1.551)


******
[TEST]
******
RMSE: 17.533 (29.309)
MAE: 14.008 (22.866)
MAPE: 0.304 (0.503)
R2: -214.934 (550.453)


Minutes Elapsed:  19.966233483950298


<IPython.core.display.Javascript object>

In [25]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps  RMSE Test  \
                                                                    mean   
0  Chemical + CS3  LSTM  Blocking Time Series Split         1  15.330693   
1  Chemical + CS3  LSTM  Blocking Time Series Split         7  10.366929   
2  Chemical + CS3  LSTM  Blocking Time Series Split        14  17.532902   
3  Chemical + CS3  LSTM              Repeated KFold         1   2.156191   
4  Chemical + CS3  LSTM              Repeated KFold         7   2.121615   
5  Chemical + CS3  LSTM              Repeated KFold        14   2.544401   

               MAE Test            MAPE Test               R2 Test              
         std       mean        std      mean       std        mean         std  
0  13.805717  11.592912  10.341854  0.243820  0.230329  -65.581502  116.275239  
1  12.787915   7.971541   9.574474  0.173492  0.212663  -37.862683   79.632283  
2  29.308839  14.008447  22.865713  0.304500  0.503295 -214.933816  550.453301  
3   0.115033   1.694980   0.079981  0.034336  0.001540    0.714076    0.038272  
4   0.193009   1.631287   0.154577  0.033145  0.002986    0.726855    0.044942  
5   0.123213   1.995277   0.099470  0.039514  0.001851    0.577271    0.068931

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [26]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.455 (1.173)
MAE: 1.859 (0.793)
MAPE: 0.038 (0.017)
R2: -0.608 (2.501)


******
[TEST]
******
RMSE: 6.153 (3.785)
MAE: 4.591 (2.503)
MAPE: 0.097 (0.058)
R2: -5.472 (6.766)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.567 (0.212)
MAE: 1.246 (0.185)
MAPE: 0.026 (0.004)
R2: 0.664 (0.211)


******
[TEST]
******
RMSE: 4.462 (2.075)
MAE: 3.231 (1.083)
MAPE: 0.068 (0.025)
R2: -2.933 (6.736)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.587 (0.210)
MAE: 1.269 (0.206)
MAPE: 0.026 (0.004)
R2: 0.606 (0.344)


******
[TEST]
******
RMSE: 5.620 (4.568)
MAE: 4.164 (3.136)
MAPE: 0.089 (0.071)
R2: -6.236 (15.820)


Minutes Elapsed:  59.853045443693794


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps  RMSE Test  \
                                                                    mean   
0  Chemical + CS3  LSTM  Blocking Time Series Split         1  15.330693   
1  Chemical + CS3  LSTM  Blocking Time Series Split         7  10.366929   
2  Chemical + CS3  LSTM  Blocking Time Series Split        14  17.532902   
3  Chemical + CS3  LSTM              Repeated KFold         1   2.156191   
4  Chemical + CS3  LSTM              Repeated KFold         7   2.121615   
5  Chemical + CS3  LSTM              Repeated KFold        14   2.544401   
6  Chemical + CS3  LSTM           Time Series Split         1   6.153107   
7  Chemical + CS3  LSTM           Time Series Split         7   4.462371   
8  Chemical + CS3  LSTM           Time Series Split        14   5.620040   

               MAE Test            MAPE Test               R2 Test              
         std       mean        std      mean       std        mean         std  
0  13.805717  11.592912  10.341854  0.243820  0.230329  -65.581502  116.275239  
1  12.787915   7.971541   9.574474  0.173492  0.212663  -37.862683   79.632283  
2  29.308839  14.008447  22.865713  0.304500  0.503295 -214.933816  550.453301  
3   0.115033   1.694980   0.079981  0.034336  0.001540    0.714076    0.038272  
4   0.193009   1.631287   0.154577  0.033145  0.002986    0.726855    0.044942  
5   0.123213   1.995277   0.099470  0.039514  0.001851    0.577271    0.068931  
6   3.785489   4.591325   2.503118  0.096759  0.058493   -5.472485    6.766226  
7   2.075319   3.231387   1.083389  0.067739  0.025165   -2.932809    6.735896  
8   4.568410   4.164477   3.136457  0.088683  0.071341   -6.236465   15.820231

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [28]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.831 (0.061)
MAE: 1.416 (0.051)
MAPE: 0.029 (0.001)
R2: 0.764 (0.016)


******
[TEST]
******
RMSE: 4.553 (0.184)
MAE: 4.301 (0.190)
MAPE: 0.079 (0.004)
R2: -4.000 (0.408)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.635 (0.093)
MAE: 1.280 (0.077)
MAPE: 0.026 (0.002)
R2: 0.813 (0.021)


******
[TEST]
******
RMSE: 4.518 (0.213)
MAE: 4.178 (0.236)
MAPE: 0.077 (0.004)
R2: -4.083 (0.473)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.658 (0.085)
MAE: 1.315 (0.083)
MAPE: 0.027 (0.002)
R2: 0.810 (0.019)


******
[TEST]
******
RMSE: 3.917 (0.128)
MAE: 3.444 (0.201)
MAPE: 0.063 (0.004)
R2: -2.886 (0.257)


Minutes Elapsed:  17.442828031380973


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation Timesteps  RMSE Test  \
                                                                     mean   
0   Chemical + CS3  LSTM  Blocking Time Series Split         1  15.330693   
1   Chemical + CS3  LSTM  Blocking Time Series Split         7  10.366929   
2   Chemical + CS3  LSTM  Blocking Time Series Split        14  17.532902   
3   Chemical + CS3  LSTM           Out of time Split         1   4.553183   
4   Chemical + CS3  LSTM           Out of time Split         7   4.518420   
5   Chemical + CS3  LSTM           Out of time Split        14   3.916716   
6   Chemical + CS3  LSTM              Repeated KFold         1   2.156191   
7   Chemical + CS3  LSTM              Repeated KFold         7   2.121615   
8   Chemical + CS3  LSTM              Repeated KFold        14   2.544401   
9   Chemical + CS3  LSTM           Time Series Split         1   6.153107   
10  Chemical + CS3  LSTM           Time Series Split         7   4.462371   
11  Chemical + CS3  LSTM           Time Series Split        14   5.620040   

                MAE Test            MAPE Test               R2 Test  \
          std       mean        std      mean       std        mean   
0   13.805717  11.592912  10.341854  0.243820  0.230329  -65.581502   
1   12.787915   7.971541   9.574474  0.173492  0.212663  -37.862683   
2   29.308839  14.008447  22.865713  0.304500  0.503295 -214.933816   
3    0.184154   4.301368   0.189808  0.079168  0.003501   -4.000121   
4    0.212742   4.178301   0.236202  0.077026  0.004381   -4.083381   
5    0.127973   3.444056   0.200588  0.063482  0.003761   -2.886425   
6    0.115033   1.694980   0.079981  0.034336  0.001540    0.714076   
7    0.193009   1.631287   0.154577  0.033145  0.002986    0.726855   
8    0.123213   1.995277   0.099470  0.039514  0.001851    0.577271   
9    3.785489   4.591325   2.503118  0.096759  0.058493   -5.472485   
10   2.075319   3.231387   1.083389  0.067739  0.025165   -2.932809   
11   4.568410   4.164477   3.136457  0.088683  0.071341   -6.236465   

                
           std  
0   116.275239  
1    79.632283  
2   550.453301  
3     0.407605  
4     0.472896  
5     0.256607  
6     0.038272  
7     0.044942  
8     0.068931  
9     6.766226  
10    6.735896  
11   15.820231

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [30]:
path = "../../../../../../reports/results/local_models/203/av/full/"
filename = f"lstm_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [31]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/203/av/grouped/"
filename = f"lstm_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>